In [1]:
import httpx
from bs4 import BeautifulSoup

SITEMAP_RU_ORIGINS = "https://minecraft.fandom.com/ru/sitemap-newsitemapxml-index.xml"
FILE_NAME = "dataset.txt"

def create_ul_text(content):
    text = ""
    rows = content.find_all("li")
    for i in range(len(rows)):
        if len(rows[i].text) > 20 and not rows[i].find("ul"):
            if len(rows[i].text) > 200:
                text += f"{i+1}. {rows[i].text[:200]}\n"
            else:
                text += f"{i+1}. {rows[i].text}\n"
    return text

def perform_url_scrapping(url):
    page = httpx.get(url)
    contents = BeautifulSoup(page.text).find_all(["p","ul"])
    pure_texts = []
    for i in range(len(contents)):
        if contents[i].name == "p" and contents[i+1].name == "ul":
            text = contents[i].text
            ul_text = create_ul_text(contents[i+1])
            pure_texts.append(text+ul_text)
        elif contents[i].name == "p" and len(contents[i].text) > 100:
            text = contents[i].text
            pure_texts.append(text)
    return pure_texts
    # returns list of texts

def main():
    r = httpx.get(SITEMAP_RU_ORIGINS)
    f = open(FILE_NAME,"w",encoding="utf-8")
    f.write("<SEP>")
    if r.status_code != httpx.codes.OK:
        print("Unreachable sitemap origins")
        return
    try:
        bs_origin = BeautifulSoup(r.text,"xml")
        sitemaps = bs_origin.find_all("sitemap")
        if not sitemaps:
            print(f"Something wrong with sitemaps\nAdditional INFO:\nbs_origin: {bs_origin}\nresponse text:{r.text}")
        for sitemap in sitemaps:
            try:
                r1 = httpx.get(sitemap.loc.text)
                print(f"USING SITEMAP\n{sitemap.loc.text}\nUSING SITEMAP")
                bs_cur = BeautifulSoup(r1.text,"xml")
                links = bs_cur.find_all("url")
                assert links, "Current sitemap has no links, is it right?"
            except KeyboardInterrupt:
                print("Parser Interrupted. Saving parsed data to file.")
                f.close()
                return
            except:
                print(f"ERROR while processing sitemap: {sitemap}")
            else:
                for link in links:
                    print(f"LINK: {link}")
                    try:
                        res = perform_url_scrapping(link.loc.text)
                    except KeyboardInterrupt:
                        print("Parser Interrupted. Saving parsed data to file.")
                        f.close()
                        return
                    except:
                        print(f"ERROR while scrapping {link.loc.text}")
                    else:
                        for text in res:
                            f.write(text)
                            f.write("<SEP>")
                        # здесь выполняем сохранение
                        # использовать сначала txt, потом Qdrant? или всё хранить в Qdrant?
    except KeyboardInterrupt:
        print("Parser Interrupted. Saving parsed data to file.")
    # except:
    #     print("Unexpected err with sitemaps parsing occured")
    finally:
        f.close()


if __name__ == "__main__":
    main()

Выходные данные были обрезаны до нескольких последних строк (5000).
</url>
LINK: <url>
<loc>https://minecraft.fandom.com/ru/wiki/%D0%9D%D0%B5%D0%BE%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0%D0%BD%D0%BD%D0%B0%D1%8F_%D0%BC%D0%B5%D0%B4%D1%8C</loc>
<lastmod>2025-03-11T11:21:12Z</lastmod>
<priority>1.0</priority>
</url>
LINK: <url>
<loc>https://minecraft.fandom.com/ru/wiki/%D0%9D%D0%B5%D0%BE%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0%D0%BD%D0%BD%D1%8B%D0%B9_%D0%BC%D0%B5%D1%82%D0%B0%D0%BB%D0%BB</loc>
<lastmod>2023-08-12T16:53:47Z</lastmod>
<priority>1.0</priority>
</url>
LINK: <url>
<loc>https://minecraft.fandom.com/ru/wiki/Beta_1.17.0.52_(Bedrock_Edition)</loc>
<lastmod>2025-04-20T16:28:38Z</lastmod>
<priority>1.0</priority>
</url>
LINK: <url>
<loc>https://minecraft.fandom.com/ru/wiki/21w17a_(Java_Edition)</loc>
<lastmod>2025-03-27T20:53:48Z</lastmod>
<priority>1.0</priority>
</url>
LINK: <url>
<loc>https://minecraft.fandom.com/ru/wiki/Thermal_Expansion_2/%D0%A1%D0%B2%D0%B5%D1%82%D0%B8%D0%BB